In [22]:
%cd /content
!rm -rf Gradu
!git clone https://github.com/Elkkujou/Gradu.git
%cd /content/Gradu
!ls

/content
Cloning into 'Gradu'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 294 (delta 52), reused 43 (delta 43), pack-reused 232 (from 2)
Receiving objects: 100% (294/294), 38.47 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (133/133), done.
/content/Gradu
 chatti_RF.ipynb	       MSCI_XGBOOST.ipynb		    RF_Gradu.ipynb
 Fama_french_XGBOOST.ipynb     Regiimi_prediction.ipynb		   'RF REGIIMI HYVÄ TRAINING.ipynb'
'Financial turbulence.ipynb'   regime_prediction_famafrench.ipynb  'RF_regime (3).ipynb'
 FT_source.xlsx		       regime_prediction_msci.ipynb	    THE_ONE.xlsx
 Gradient_boost_malli.ipynb    regime_pred.txt


In [23]:
calc_drawdowns = True
drawdowns_from = 'USA MOMENTUM Standard (Large+Mid Cap)'
cluster_n = 2

normmalize = True
ML_reg = True
ML_long = False
DL = False

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from tabulate import tabulate

from IPython.display import display, HTML
from sklearn.cluster import KMeans

In [25]:
xls_file = pd.ExcelFile("/content/Gradu/THE_ONE.xlsx")
excel_df = xls_file.parse("returns non-log")

excel_df.columns = excel_df.columns.get_level_values(0)
excel_df['Date'] = pd.to_datetime(excel_df['Date'])
excel_df.sort_values('Date', inplace=True)

print("Headers in the 'returns non-log' sheet:")
print(excel_df.columns)

Headers in the 'returns non-log' sheet:
Index(['Date', 'USA Standard (Large+Mid Cap)',
       'USA MOMENTUM Standard (Large+Mid Cap)',
       'USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap)',
       'USA RISK WEIGHTED Standard (Large+Mid Cap)',
       'USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap)',
       'USA ENHANCED VALUE Standard (Large+Mid Cap)', 'Winning Factor',
       'VIXCLS', 'GDPC1', 'CPI', 'LEI', '10yr', 'Financial Turbulance',
       '(Regime)', 'Financial condition', 'Drawdown', 'HY Spread', 'IG Spread',
       '10y-2y', 'Vix-regimes2', 'Vix-regimes3', 'FT_regimes2', 'FT_regimes3',
       'DD_regimes2', 'DD_regimes3', 'cluster_0 (minimal drawdown',
       'cluster_1 (moderate drawdown)', 'cluster_2 (severe drawdown)', 'SMB',
       'HML', 'RMW', 'CMA', 'MOM', '(Regimes)', 'Regimes'],
      dtype='object')


In [26]:
if calc_drawdowns:
    # Calculate the cumulative portfolio values using excel_df and the specified drawdowns_from column
    portfolio = (1 + excel_df[drawdowns_from]).cumprod()

    # Define a function to compute the maximum drawdown over a rolling window
    def max_dd(s):
        return (s / s.cummax() - 1).min()

    # Calculate the 3-month rolling drawdown using the portfolio series
    dd_series = portfolio.rolling(window=3, min_periods=3).apply(max_dd, raw=False)

    # Create a temporary DataFrame to store dates and the calculated drawdowns
    dd_temp = pd.DataFrame({
        'Date': excel_df['Date'],
        '3M_Max_Drawdown': dd_series
    })

    # Drop rows with missing drawdown data to ensure complete observations
    dd_temp = dd_temp.dropna()

    print("Drawdown calculation (temporary DataFrame):")
    print(dd_temp.head())


Drawdown calculation (temporary DataFrame):
        Date  3M_Max_Drawdown
2 1990-05-31        -0.014564
3 1990-06-29         0.000000
4 1990-07-31         0.000000
5 1990-08-31        -0.086261
6 1990-09-28        -0.128945


In [27]:
# Fit KMeans clustering on the temporary DataFrame's '3M_Max_Drawdown' column
kmeans = KMeans(n_clusters=cluster_n, random_state=42)
dd_temp['DD_cluster'] = kmeans.fit_predict(dd_temp[['3M_Max_Drawdown']])

# Merge the cluster labels from dd_temp into excel_df by matching on the 'Date' column.
# This adds a new column 'DD_cluster' to excel_df.
excel_df = excel_df.merge(dd_temp[['Date', 'DD_cluster']], on='Date', how='left')

print("excel_df with DD_cluster added:")
from IPython.display import display, HTML
display(HTML(excel_df.head().to_html(index=False)))


excel_df with DD_cluster added:


<ipython-input-27-ae5c710a1ef9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd_temp['DD_cluster'] = kmeans.fit_predict(dd_temp[['3M_Max_Drawdown']])


Date,USA Standard (Large+Mid Cap),USA MOMENTUM Standard (Large+Mid Cap),USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap),USA RISK WEIGHTED Standard (Large+Mid Cap),USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),USA ENHANCED VALUE Standard (Large+Mid Cap),Winning Factor,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,(Regime),Financial condition,Drawdown,HY Spread,IG Spread,10y-2y,Vix-regimes2,Vix-regimes3,FT_regimes2,FT_regimes3,DD_regimes2,DD_regimes3,cluster_0 (minimal drawdown,cluster_1 (moderate drawdown),cluster_2 (severe drawdown),SMB,HML,RMW,CMA,MOM,(Regimes),Regimes,DD_cluster
1990-03-30,0.020702,0.028022,0.012313,0.013039,0.036524,0.013845,USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),21.40,0.003630,0.004687,63.5,8.59,9.383220,Recovery,0.04713,0.000000,NaN,NaN,0.08,High,Low,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,0.0162,-0.0292,0.0204,-0.0102,0.0180,Normal,Normal,NaN
1990-04-30,-0.023292,-0.014564,-0.031345,-0.042294,-0.007806,-0.032085,USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),18.10,0.003630,0.002333,63.6,8.79,4.756642,Slowdown,-0.13602,-0.023292,NaN,NaN,0.10,High,High,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0033,-0.0259,0.0178,-0.0098,0.0239,Normal,Normal,NaN
1990-05-31,0.089444,0.112070,0.079487,0.080355,0.101743,0.081134,USA MOMENTUM Standard (Large+Mid Cap),16.82,0.003630,0.001552,63.3,8.76,14.686352,Slowdown,-0.15632,0.000000,NaN,NaN,0.19,High,High,Normal Turbulence,High Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0235,-0.0383,0.0157,-0.0168,0.0303,Normal,Normal,0.0
1990-06-29,-0.006432,0.022340,-0.010553,-0.010138,0.011909,-0.016679,USA MOMENTUM Standard (Large+Mid Cap),18.39,0.000666,0.006197,63.3,8.48,8.020190,Slowdown,0.08472,-0.006432,NaN,NaN,0.45,High,High,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,0.0135,-0.0193,-0.0103,-0.0039,0.0240,Normal,Normal,0.0
1990-07-31,-0.004276,0.004471,-0.001134,-0.011361,-0.000470,-0.008691,USA MOMENTUM Standard (Large+Mid Cap),28.18,0.000666,0.004619,63.3,8.47,25.637526,Slowdown,0.24339,-0.010680,NaN,NaN,0.79,Normal,Low,High Turbulence,High Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0316,-0.0003,-0.0018,0.0321,0.0610,Normal,Normal,0.0


In [28]:
#FEATURES = ['VIXCLS', 'CPI', 'LEI', '10yr', 'Financial condition', '10y-2y', 'Financial Turbulance']

FEATURES = [
    'VIXCLS',
    'GDPC1',
    'CPI',
    'LEI',
    '10yr',
    'Financial Turbulance',
    'Financial condition',
    'HY Spread',
    'IG Spread',
    '10y-2y'
]


TRUE_REGIME = 'DD_cluster'
DATE = 'Date'

selected_columns =  [DATE] + [TRUE_REGIME] +FEATURES


In [29]:
missing = [col for col in selected_columns if col not in excel_df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

# Subset and drop rows with empty values
input_df = excel_df[selected_columns]
orig_rows = len(input_df)
input_df = input_df.dropna()
print(f"Original rows: {orig_rows}, Dropped rows: {orig_rows - len(input_df)}")
print(f"First obs: {input_df[DATE].min()}, Last obs: {input_df[DATE].max()}")

# Display as neat HTML table (adjust 'rows_to_show' as needed)
rows_to_show = 10
from IPython.display import display, HTML
display(HTML(input_df.head(rows_to_show).to_html(index=False)))

Original rows: 416, Dropped rows: 81
First obs: 1996-12-31 00:00:00, Last obs: 2024-10-31 00:00:00


Date,DD_cluster,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,Financial condition,HY Spread,IG Spread,10y-2y
1996-12-31,0.0,19.47,0.006454,0.002520,77.7,6.30,7.860576,0.00043,3.04,0.61,0.59
1997-01-31,0.0,20.14,0.006454,0.001886,77.8,6.58,14.956959,-0.03500,2.73,0.59,0.47
1997-02-28,0.0,20.17,0.006454,0.001882,78.3,6.42,13.043063,0.05391,2.84,0.57,0.47
1997-03-31,1.0,19.66,0.016652,0.000626,79.3,6.69,10.821978,0.05726,2.94,0.59,0.43
1997-04-30,1.0,19.92,0.016652,0.000626,79.7,6.89,10.938933,0.00925,2.68,0.61,0.45
1997-05-30,0.0,20.19,0.016652,0.000000,79.9,6.71,4.038058,-0.01056,2.67,0.58,0.43
1997-06-30,0.0,20.53,0.012486,0.001876,80.4,6.49,7.517614,-0.02150,2.71,0.56,0.28
1997-07-31,0.0,23.08,0.012486,0.001248,81.1,6.22,14.684361,0.01571,2.59,0.57,0.37
1997-08-29,1.0,23.81,0.012486,0.002494,82.0,6.30,10.252527,0.03394,2.59,0.54,0.32
1997-09-30,1.0,23.87,0.008539,0.002488,82.3,6.21,6.511195,0.10651,2.99,0.67,0.21


In [30]:
from tabulate import tabulate

# Calculate the number of observations for each regime in the data
regime_counts = input_df[TRUE_REGIME].value_counts().sort_index().reset_index()
regime_counts.columns = [TRUE_REGIME, 'Count']

print("Number of observations per regime:")
print(tabulate(regime_counts, headers='keys', tablefmt='psql', showindex=False))


Number of observations per regime:
+--------------+---------+
|   DD_cluster |   Count |
|--------------+---------|
|            0 |     258 |
|            1 |      77 |
+--------------+---------+


In [31]:
if normmalize == True:
    from sklearn.preprocessing import StandardScaler

    # Instantiate the scaler and apply it in-place to the FEATURES columns
    scaler = StandardScaler()
    input_df[FEATURES] = scaler.fit_transform(input_df[FEATURES])

    print("Normalization applied. First 10 observations:")
    display(HTML(input_df.head(10).to_html(index=False)))


Normalization applied. First 10 observations:


Date,DD_cluster,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,Financial condition,HY Spread,IG Spread,10y-2y
1996-12-31,0.0,-0.110862,0.029919,0.158638,-1.496475,1.871825,-0.411545,-0.000050,-0.909269,-1.082181,-0.415568
1997-01-31,0.0,-0.025812,0.029919,-0.058714,-1.487664,2.062500,0.235650,-0.253834,-1.032720,-1.106567,-0.539840
1997-02-28,0.0,-0.022004,0.029919,-0.059929,-1.443610,1.953543,0.061101,0.383025,-0.988915,-1.130954,-0.539840
1997-03-31,1.0,-0.086743,0.867842,-0.489887,-1.355504,2.137408,-0.141463,0.407021,-0.949092,-1.106567,-0.581264
1997-04-30,1.0,-0.053739,0.867842,-0.490021,-1.320261,2.273604,-0.130797,0.063127,-1.052631,-1.082181,-0.560552
1997-05-30,0.0,-0.019465,0.867842,-0.704261,-1.302640,2.151027,-0.760162,-0.078771,-1.056614,-1.118761,-0.581264
1997-06-30,0.0,0.023694,0.525492,-0.061944,-1.258586,2.001211,-0.442824,-0.157134,-1.040684,-1.143147,-0.736604
1997-07-31,0.0,0.347390,0.525492,-0.276851,-1.196911,1.817346,0.210789,0.109400,-1.088472,-1.130954,-0.643400
1997-08-29,1.0,0.440056,0.525492,0.149492,-1.117615,1.871825,-0.193398,0.239981,-1.088472,-1.167534,-0.695180
1997-09-30,1.0,0.447673,0.201170,0.147369,-1.091183,1.810537,-0.534610,0.759796,-0.929180,-1.009021,-0.809096


In [39]:
if ML_reg == True:

    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score, confusion_matrix

    # Define predictors and target using the given column lists
    predictors = FEATURES
    target = TRUE_REGIME

    # Drop rows with missing values in predictors or target
    data = input_df.dropna(subset=predictors + [target]).reset_index(drop=True)
    X = data[predictors]
    y = data[target]
    # Also grab the DATE column for printing prediction timestamps
    dates = data[DATE]

    # Define the base models with high parameters, now including Gradient Boosting
    base_models = {
        'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42),
        'GradientBoosting': GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42),
        'SVC': SVC(probability=True, C=1000, kernel='rbf', gamma='scale'),
        'GaussianNB': GaussianNB(var_smoothing=1e-8)
    }

    # Define the meta-learner
    meta_learner = LogisticRegression(max_iter=1000, random_state=42)

    window_size = 60  # fixed window length once reached
    min_data_pct = 0.2  # start base model predictions when available data >= 80% of window_size
    base_threshold = int(window_size * min_data_pct)

    # Initialize dictionaries/lists to store rolling accuracies and meta weights
    rolling_accuracies_base = {name: [] for name in base_models}
    rolling_accuracies_meta = []
    meta_weights_list = []  # record meta-learner coefficients

    # Initialize expanding meta training set (meta features and outcomes)
    meta_train_features = []
    meta_train_targets = []

    print("Starting rolling window evaluation with stacking ensemble and expanding meta learner:")

    for i in range(len(X)):
        # Do not make any prediction until we have at least base_threshold observations
        if i < base_threshold:
            print(f"Skipping prediction for timestamp {dates.iloc[i]} "
                  f"(insufficient data: {i} observations, need {base_threshold}).")
            continue

        # For base models, if i is less than full window_size, use all data from start to i.
        # Once i >= window_size, use a fixed rolling window of size window_size.
        if i < window_size:
            X_train_window = X.iloc[:i]
            y_train_window = y.iloc[:i]
        else:
            X_train_window = X.iloc[i-window_size:i]
            y_train_window = y.iloc[i-window_size:i]

        X_test_window = X.iloc[[i]]
        y_test_window = y.iloc[[i]]
        test_date = dates.iloc[i]

        # --- Step 1: Generate meta features from base model predictions ---
        meta_features_test = np.zeros((1, len(base_models)))
        for j, (name, model) in enumerate(base_models.items()):
            model.fit(X_train_window, y_train_window)
            probs = model.predict_proba(X_test_window)
            if probs.shape[1] == 1:
                # Edge-case: model returns only one class probability
                y_pred_prob = 1.0 if model.classes_[0] == 1 else 0.0
            else:
                y_pred_prob = probs[0, 1]
            meta_features_test[0, j] = y_pred_prob

            # Record accuracy for each base model on this test observation
            y_pred = model.predict(X_test_window)[0]
            acc = accuracy_score(y_test_window, [y_pred])
            rolling_accuracies_base[name].append(acc)

        # --- Step 2: Use meta training set to make meta prediction once full window is reached ---
        if i >= window_size:
            meta_learner.fit(np.array(meta_train_features), np.array(meta_train_targets))
            meta_pred = meta_learner.predict(meta_features_test)[0]
            meta_acc = accuracy_score(y_test_window, [meta_pred])
            rolling_accuracies_meta.append(meta_acc)
            meta_weights_list.append(meta_learner.coef_[0])
            prediction_source = "Meta Learner"
            print(f"Test date {test_date}: {prediction_source} Prediction = {meta_pred}, Accuracy = {meta_acc:.2f}")
        else:
            # For i in [base_threshold, window_size), only base model predictions are available
            print(f"Test date {test_date}: Base Model predictions only (growing training window).")

        # --- Step 3: Append current meta features and true outcome to the expanding meta training set ---
        meta_train_features.append(meta_features_test[0])
        meta_train_targets.append(y_test_window.iloc[0])

    # --- Aggregation of Results ---
    avg_base_acc = {name: np.mean(acc_list) if acc_list else 0 for name, acc_list in rolling_accuracies_base.items()}
    avg_meta_acc = np.mean(rolling_accuracies_meta) if rolling_accuracies_meta else 0
    avg_meta_weights = np.mean(meta_weights_list, axis=0) if meta_weights_list else np.array([])

    results_table = pd.DataFrame({
        'Model': list(avg_base_acc.keys()) + ['Meta Learner'],
        'Average Accuracy': list(avg_base_acc.values()) + [avg_meta_acc]
    })
    print("\nRolling Window Average Accuracies:")
    print(results_table)

    if avg_meta_weights.size > 0:
        weights_table = pd.DataFrame({
            'Base Model': list(base_models.keys()),
            'Average Weight': avg_meta_weights
        })
        print("\nAggregate Meta-Learner Weights (Average Coefficients):")
        print(weights_table)


Starting rolling window evaluation with stacking ensemble and expanding meta learner:
Skipping prediction for timestamp 1996-12-31 00:00:00 (insufficient data: 0 observations, need 12).
Skipping prediction for timestamp 1997-01-31 00:00:00 (insufficient data: 1 observations, need 12).
Skipping prediction for timestamp 1997-02-28 00:00:00 (insufficient data: 2 observations, need 12).
Skipping prediction for timestamp 1997-03-31 00:00:00 (insufficient data: 3 observations, need 12).
Skipping prediction for timestamp 1997-04-30 00:00:00 (insufficient data: 4 observations, need 12).
Skipping prediction for timestamp 1997-05-30 00:00:00 (insufficient data: 5 observations, need 12).
Skipping prediction for timestamp 1997-06-30 00:00:00 (insufficient data: 6 observations, need 12).
Skipping prediction for timestamp 1997-07-31 00:00:00 (insufficient data: 7 observations, need 12).
Skipping prediction for timestamp 1997-08-29 00:00:00 (insufficient data: 8 observations, need 12).
Skipping predi

In [ ]:
if ML == True:

    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score, confusion_matrix

    # Define predictors and target using the given column lists
    predictors = FEATURES
    target = TRUE_REGIME

    # Drop rows with missing values in predictors or target
    data = input_df.dropna(subset=predictors + [target]).reset_index(drop=True)
    X = data[predictors]
    y = data[target]
    # Also grab the DATE column for printing prediction timestamps
    dates = data[DATE]

    # Define the base models with high parameters
    base_models = {
        'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42),
        'SVC': SVC(probability=True, C=1000, kernel='rbf', gamma='scale'),
        'GaussianNB': GaussianNB(var_smoothing=1e-8)
    }

    # Define the meta-learner
    meta_learner = LogisticRegression(max_iter=1000, random_state=42)

    window_size = 60  # e.g., 60 observations for training
    meta_start_threshold = window_size  # start meta predictions after 60 meta samples

    # Initialize dictionaries/lists to store rolling accuracies
    rolling_accuracies_base = {name: [] for name in base_models}
    rolling_accuracies_meta = []
    meta_weights_list = []  # store meta-learner coefficients when trained

    # Initialize expanding meta training set (meta features and outcomes)
    meta_train_features = []
    meta_train_targets = []

    print("Starting rolling window evaluation with stacking ensemble and expanding meta learner:")

    # Loop over the dataset in a rolling window manner
    for i in range(len(X)):
        if i < window_size:
            print(f"Skipping prediction for timestamp {dates.iloc[i]} "
                  f"(insufficient data: {i} observations, need {window_size}).")
            continue

        # Define training set as the previous window_size observations and test as the next point
        X_train_window = X.iloc[i-window_size:i]
        y_train_window = y.iloc[i-window_size:i]
        X_test_window = X.iloc[[i]]
        y_test_window = y.iloc[[i]]
        test_date = dates.iloc[i]

        # --- Step 1: Get meta features from base models for the test observation ---
        meta_features_test = np.zeros((1, len(base_models)))
        for j, (name, model) in enumerate(base_models.items()):
            model.fit(X_train_window, y_train_window)
            probs = model.predict_proba(X_test_window)
            if probs.shape[1] == 1:
                # Handle edge-case when model predicts only one class
                y_pred_prob = 1.0 if model.classes_[0] == 1 else 0.0
            else:
                y_pred_prob = probs[0, 1]
            meta_features_test[0, j] = y_pred_prob

            # Record base model accuracy on this test observation
            y_pred = model.predict(X_test_window)[0]
            acc = accuracy_score(y_test_window, [y_pred])
            rolling_accuracies_base[name].append(acc)

        # --- Step 2: Use expanding meta training set to predict final outcome ---
        if len(meta_train_features) >= meta_start_threshold:
            # Train the meta-learner on all previously accumulated meta samples
            meta_learner.fit(np.array(meta_train_features), np.array(meta_train_targets))
            meta_pred = meta_learner.predict(meta_features_test)[0]
            meta_acc = accuracy_score(y_test_window, [meta_pred])
            rolling_accuracies_meta.append(meta_acc)
            meta_weights_list.append(meta_learner.coef_[0])
            print(f"Test date {test_date}: Meta Learner Prediction = {meta_pred}, Accuracy = {meta_acc:.2f}")
        else:
            print(f"Test date {test_date}: Not enough meta training samples for meta learner prediction (have {len(meta_train_features)}).")

        # --- Step 3: Append current meta features and true outcome to the expanding meta training set ---
        meta_train_features.append(meta_features_test[0])
        meta_train_targets.append(y_test_window.iloc[0])

    # --- Aggregation of results ---
    # Compute average accuracy for each base model and meta-learner
    avg_base_acc = {name: np.mean(acc_list) if acc_list else 0 for name, acc_list in rolling_accuracies_base.items()}
    avg_meta_acc = np.mean(rolling_accuracies_meta) if rolling_accuracies_meta else 0
    avg_meta_weights = np.mean(meta_weights_list, axis=0) if meta_weights_list else np.array([])

    # Create a table for accuracies
    results_table = pd.DataFrame({
        'Model': list(avg_base_acc.keys()) + ['Meta Learner'],
        'Average Accuracy': list(avg_base_acc.values()) + [avg_meta_acc]
    })
    print("\nRolling Window Average Accuracies:")
    print(results_table)

    # Create a table for aggregate meta-learner weights (if available)
    if avg_meta_weights.size > 0:
        weights_table = pd.DataFrame({
            'Base Model': list(base_models.keys()),
            'Average Weight': avg_meta_weights
        })
        print("\nAggregate Meta-Learner Weights (Average Coefficients):")
        print(weights_table)


In [ ]:
if DL == True:
    import warnings
    warnings.filterwarnings("ignore")  # Suppress warnings

    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
    import xgboost as xgb
    from sklearn.metrics import confusion_matrix, accuracy_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import pandas as pd
    from IPython.display import display, HTML

    # Define the experiment function with remapping of training labels if needed
    def run_experiment(training_window, seq_len,
                       lstm_units_first=128, lstm_units_second=32, dropout_rate=0.3,
                       xgb_n_estimators=100, xgb_max_depth=4, xgb_learning_rate=0.1):
        """
        Runs the rolling-window hybrid (LSTM + XGBoost) experiment using a given training_window (in months)
        and a sequence length (seq_len). If the training labels do not form a contiguous set starting at 0,
        they are remapped. Returns overall accuracy, total number of predictions, and regime-specific accuracies.
        """
        all_preds = []   # To store predictions
        all_true = []    # To store true regime labels

        # Ensure data is sorted by date
        data = input_df.sort_values(DATE).reset_index(drop=True)

        # Rolling window: iterate from index=training_window to end of data
        for i in range(training_window, len(data)):
            # Use the previous 'training_window' months as training data
            train_window = data.iloc[i - training_window:i].reset_index(drop=True)
            if len(train_window) < training_window:
                continue

            # Generate overlapping sequences of length 'seq_len' within the training window
            X_train_sequences = []
            y_train = []
            for j in range(len(train_window) - seq_len + 1):
                seq = train_window[FEATURES].iloc[j : j + seq_len].values
                target = train_window[TRUE_REGIME].iloc[j + seq_len - 1]  # regime at end of sequence
                X_train_sequences.append(seq)
                y_train.append(target)
            X_train_sequences = np.array(X_train_sequences)  # shape: (num_samples, seq_len, num_features)
            y_train = np.array(y_train)

            # Remap training labels to contiguous integers starting at 0 if needed.
            unique_classes = np.unique(y_train)
            mapping = {old: new for new, old in enumerate(unique_classes)}
            inverse_mapping = {v: k for k, v in mapping.items()}
            y_train_mapped = np.array([mapping[val] for val in y_train])

            # Build the LSTM feature extractor using an explicit Input layer
            input_layer = Input(shape=(seq_len, len(FEATURES)))
            lstm_out = LSTM(lstm_units_first, return_sequences=True)(input_layer)
            lstm_out = LSTM(lstm_units_second, return_sequences=False)(lstm_out)
            dropout_out = Dropout(dropout_rate)(lstm_out)
            feature_extractor = Model(inputs=input_layer, outputs=dropout_out)

            # Extract features from training sequences
            X_train_features = feature_extractor.predict(X_train_sequences, verbose=0)

            # Train an XGBoost classifier on these features using the remapped labels.
            clf = xgb.XGBClassifier(
                n_estimators=xgb_n_estimators,
                max_depth=xgb_max_depth,
                learning_rate=xgb_learning_rate,
                use_label_encoder=False,
                eval_metric='mlogloss'
            )
            clf.fit(X_train_features, y_train_mapped)

            # For prediction, use the last 'seq_len'-month sequence from the training window
            X_pred_seq = train_window[FEATURES].iloc[-seq_len:].values.reshape(1, seq_len, len(FEATURES))
            X_pred_feature = feature_extractor.predict(X_pred_seq, verbose=0)
            pred_mapped = clf.predict(X_pred_feature)[0]
            # Map back to original label
            pred_regime = inverse_mapping[pred_mapped]

            # The true regime for the next month (at time index i)
            true_regime = data[TRUE_REGIME].iloc[i]

            all_preds.append(pred_regime)
            all_true.append(true_regime)

        # Compute overall accuracy and total number of predictions
        overall_acc = accuracy_score(all_true, all_preds)
        total_obs = len(all_preds)

        # Compute regime-specific accuracies
        regimes = np.unique(all_true)
        regime_accuracies = {}
        for r in regimes:
            indices = [idx for idx, val in enumerate(all_true) if val == r]
            if len(indices) > 0:
                correct = sum(1 for idx in indices if all_preds[idx] == all_true[idx])
                regime_acc = correct / len(indices)
                regime_accuracies[r] = regime_acc
            else:
                regime_accuracies[r] = np.nan

        return {
            "training_window": training_window,
            "seq_len": seq_len,
            "total_obs": total_obs,
            "overall_accuracy": overall_acc,
            "regime_accuracies": regime_accuracies
        }

    # Prepare to store experiment results in a list of dictionaries
    results = []

    # Define training windows from 1 to 10 years (in months) – assume 1 year = 12 months
    for tw in range(12, 121, 12):  # 12, 24, ..., 120 months
        # Define sequence length percentages: 25%, 50%, 75%, and 100% of training window
        for frac in [0.25, 0.5, 0.75, 1.0]:
            seq = int(round(tw * frac))
            print("="*50)
            print(f"Running experiment with training_window = {tw} months and seq_len = {seq} months")
            res = run_experiment(training_window=tw, seq_len=seq)
            # Build a result row that includes run parameters and accuracies
            row = {
                "Training Window (months)": tw,
                "Seq Length (months)": seq,
                "Total Observations": res["total_obs"],
                "Overall Accuracy": f"{res['overall_accuracy']*100:.2f}%",
                "Accuracy Regime 0": f"{res['regime_accuracies'].get(0, np.nan)*100:.2f}%" if 0 in res["regime_accuracies"] else "N/A",
                "Accuracy Regime 1": f"{res['regime_accuracies'].get(1, np.nan)*100:.2f}%" if 1 in res["regime_accuracies"] else "N/A",
                "Accuracy Regime 2": f"{res['regime_accuracies'].get(2, np.nan)*100:.2f}%" if 2 in res["regime_accuracies"] else "N/A"
            }
            results.append(row)
            print("="*50)

    # Convert results to a DataFrame and display as an HTML table
    df_results = pd.DataFrame(results)
    display(HTML(df_results.to_html(index=False)))


In [ ]:
# Convert results to a DataFrame and display as an HTML table
df_results = pd.DataFrame(results)
display(HTML(df_results.to_html(index=False)))
